In [7]:
import os
import warnings
from dotenv import load_dotenv, find_dotenv
from crewai import Agent, Task, Crew, Process
from crewai_tools import (
  FileReadTool,
  ScrapeWebsiteTool,
  MDXSearchTool,
  SerperDevTool,
  PDFSearchTool
)

warnings.filterwarnings('ignore')

load_dotenv(find_dotenv())


True

In [8]:
openai_api_key = os.getenv("OPENAPI_API_KEY")
serper_api_key = os.getenv("SERPER_API_KEY")

os.environ["OPENAI_MODEL_NAME"] = 'gpt-4-turbo'

RESUME_PATH_MD = "./lab_assets/job_application/fake_resume.md"

### Tools

In [9]:
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()
read_resume = FileReadTool(file_path=RESUME_PATH)
semantic_search_resume = MDXSearchTool(mdx=RESUME_PATH)

In [10]:
#from IPython.display import Markdown, display
#display(Markdown(RESUME_PATH))

### Agents

In [11]:
# Agent 1: Researcher
researcher = Agent(
    role="Tech Job Researcher",
    goal="Make sure to do amazing analysis on "
         "job posting to help job applicants",
    tools = [scrape_tool, search_tool],
    verbose=True,
    backstory=(
        "As a Job Researcher, your prowess in "
        "navigating and extracting critical "
        "information from job postings is unmatched."
        "Your skills help pinpoint the necessary "
        "qualifications and skills sought "
        "by employers, forming the foundation for "
        "effective application tailoring."
    )
)

In [12]:
# Agent 2: Profiler
profiler = Agent(
    role="Personal Profiler for Engineers",
    goal="Do increditble research on job applicants "
         "to help them stand out in the job market",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "Equipped with analytical prowess, you dissect "
        "and synthesize information "
        "from diverse sources to craft comprehensive "
        "personal and professional profiles, laying the "
        "groundwork for personalized resume enhancements."
    )
)

In [13]:
# Agent 3: Resume Strategist
resume_strategist = Agent(
    role="Resume Strategist for Engineers",
    goal="Find all the best ways to make a "
         "resume stand out in the job market.",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "With a strategic mind and an eye for detail, you "
        "excel at refining resumes to highlight the most "
        "relevant skills and experiences, ensuring they "
        "resonate perfectly with the job's requirements."
    )
)

In [14]:
# Agent 4: Interview Preparer
interview_preparer = Agent(
    role="Engineering Interview Preparer",
    goal="Create interview questions and talking points "
         "based on the resume and job requirements",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "Your role is crucial in anticipating the dynamics of "
        "interviews. With your ability to formulate key questions "
        "and talking points, you prepare candidates for success, "
        "ensuring they can confidently address all aspects of the "
        "job they are applying for."
    )
)

### Tasks

In [15]:
# Task for Researcher Agent: Extract Job Requirements
research_task = Task(
    description=(
        "Analyze the job posting URL provided ({job_posting_url}) "
        "to extract key skills, experiences, and qualifications "
        "required. Use the tools to gather content and identify "
        "and categorize the requirements."
    ),
    expected_output=(
        "A structured list of job requirements, including necessary "
        "skills, qualifications, and experiences."
    ),
    agent=researcher,
    async_execution=True
)

In [16]:
# Task for Profiler Agent: Compile Comprehensive Profile
profile_task = Task(
    description=(
        "Compile a detailed personal and professional profile "
        "using the GitHub ({github_url}) URLs, and personal write-up "
        "({personal_writeup}). Utilize tools to extract and "
        "synthesize information from these sources."
    ),
    expected_output=(
        "A comprehensive profile document that includes skills, "
        "project experiences, contributions, interests, and "
        "communication style."
    ),
    agent=profiler,
    async_execution=True
)

In [17]:
# Task for Resume Strategist Agent: Align Resume with Job Requirements
resume_strategy_task = Task(
    description=(
        "Using the profile and job requirements obtained from "
        "previous tasks, tailor the resume to highlight the most "
        "relevant areas. Employ tools to adjust and enhance the "
        "resume content. Make sure this is the best resume ever but "
        "don't make up any information. Update every section, "
        "inlcuding the initial summary, work experience, skills, "
        "and education. All to better reflrect the candidates "
        "abilities and how it matches the job posting."
    ),
    expected_output=(
        "An updated resume that effectively highlights the candidate's "
        "qualifications and experiences relevant to the job."
    ),
    output_file="tailored_resume.md",
    context=[research_task, profile_task],
    agent=resume_strategist
)

In [18]:
# Task for Interview Preparer Agent: Develop Interview Materials
interview_preparation_task = Task(
    description=(
        "Create a set of potential interview questions and talking "
        "points based on the tailored resume and job requirements. "
        "Utilize tools to generate relevant questions and discussion "
        "points. Make sure to use these question and talking points to "
        "help the candiadte highlight the main points of the resume "
        "and how it matches the job posting."
    ),
    expected_output=(
        "A document containing key questions and talking points "
        "that the candidate should prepare for the initial interview."
    ),
    output_file="interview_materials.md",
    context=[research_task, profile_task, resume_strategy_task],
    agent=interview_preparer
)


### Crew

In [19]:
job_application_crew = Crew(
    agents=[researcher,
            profiler,
            resume_strategist,
            interview_preparer],

    tasks=[research_task,
           profile_task,
           resume_strategy_task,
           interview_preparation_task],

    verbose=True
)

Input Data

In [20]:
job_application_inputs = {
    'job_posting_url': 'https://jobs.lever.co/AIFund/6c82e23e-d954-4dd8-a734-c0c2c5ee00f1?lever-origin=applied&lever-source%5B%5D=AI+Fund',
    'github_url': 'https://github.com/joaomdmoura',
    'personal_writeup': """Noah is an accomplished Software
    Engineering Leader with 18 years of experience, specializing in
    managing remote and in-office teams, and expert in multiple
    programming languages and frameworks. He holds an MBA and a strong
    background in AI and data science. Noah has successfully led
    major tech initiatives and startups, proving his ability to drive
    innovation and growth in the tech industry. Ideal for leadership
    roles that require a strategic and innovative approach."""
}

In [21]:
### this execution will take a few minutes to run
result = job_application_crew.kickoff(inputs=job_application_inputs)

 [2024-08-18 17:40:56][DEBUG]: == Working Agent: Tech Job Researcher
 [2024-08-18 17:40:56][INFO]: == Starting Task: Analyze the job posting URL provided (https://jobs.lever.co/AIFund/6c82e23e-d954-4dd8-a734-c0c2c5ee00f1?lever-origin=applied&lever-source%5B%5D=AI+Fund) to extract key skills, experiences, and qualifications required. Use the tools to gather content and identify and categorize the requirements.
 [2024-08-18 17:40:56][DEBUG]: == Working Agent: Personal Profiler for Engineers
 [2024-08-18 17:40:56][INFO]: == Starting Task: Compile a detailed personal and professional profile using the GitHub (https://github.com/joaomdmoura) URLs, and personal write-up (Noah is an accomplished Software
    Engineering Leader with 18 years of experience, specializing in
    managing remote and in-office teams, and expert in multiple
    programming languages and frameworks. He holds an MBA and a strong
    background in AI and data science. Noah has successfully led
    major tech initiative

In [22]:
from IPython.display import Markdown, display
display(Markdown("./tailored_resume.md"))

# Noah Williams
- Email: noah.williams@example.dev
- Phone: +44 11 111 11111

## Profile
Noah Williams is a highly skilled Full Stack Engineer and Software Engineering Leader with 18 years of industry experience. Specializing in both front-end and back-end development, Noah has a proven track record in leading the design and implementation of scalable software solutions across diverse platforms, including desktop and mobile applications. He is proficient in HTML, CSS, JavaScript, Python, Ruby, and has significant experience with React, Angular, Node.js, and various databases like MySQL and MongoDB. His expertise also extends into AI and machine learning, making him adept at integrating cutting-edge technologies to enhance application responsiveness and efficiency.

## Work History

### DataKernel: Director of Software Engineering (remote) — 2022 - Present
- Transformed the engineering division into a key revenue generator by implementing scalable software solutions and integrating AI technologies.
- Led a cross-functional team, fostering an environment that encouraged innovation and strategic project execution, aligning with company's long-term goals.

### DataKernel: Senior Software Engineering Manager (remote) — 2019 - 2022
- Directed the full software development lifecycle, enhancing product capabilities through effective leadership and advanced technical strategies.
- Managed multi-regional teams, emphasizing transparency and continuous improvement, significantly boosting team morale and performance.

### InnovPet: Founder & CEO (remote) — 2019 - 2022
- Founded and led a startup focused on innovative IoT solutions for pet care, managing all aspects from conception to market entry, highlighting strong project management and organizational skills.
- Developed the application's front-end and managed the database using MongoDB, ensuring robust data protection and security settings.

### EliteDevs: Engineering Manager (remote) — 2018 - 2019
- Enhanced software responsiveness and efficiency by leading the development of new products using Ruby on Rails and React.
- Implemented strategic initiatives that improved inter-departmental collaboration and project outcomes.

### PrintPack: Engineering Manager (remote) — 2016 - 2018
- Spearheaded the development of predictive modeling tools and integrated data analytics into business processes, significantly improving decision-making and customer behavior analysis.

### DriveAI: Senior Software Engineer (remote) — 2015 - 2016
- Developed a central API and implemented advanced caching strategies, drastically enhancing system performance and user satisfaction.

### BetCraft: CTO — 2013 - 2015
- Led the technology strategy and development using React and Angular, post-Series A funding, driving significant technological advancement and market expansion.

## Education

### MBA in Information Technology
London Business School - MBA

### Advanced Leadership Techniques
University of London - Certification

### Data Science Specialization
Coursera (Johns Hopkins University) - Certification

### B.Sc. in Computer Science
University of Edinburgh - Bachelor’s degree

Noah's extensive experience in software development, leadership, and his technical prowess in full stack environments make him an exceptional candidate for the Full Stack Engineer position at AI Fund. His ability to drive innovation and growth, coupled with his strong communication skills and strategic insight, align perfectly with the responsibilities and expectations of the role.